In [1]:
%cd C:\Users\Mo Ragheb\Downloads\-Python-3-Bootcamp-master\Labs\Scrapper

import json
import requests
from dateutil import parser
from datetime import  datetime,timedelta
import time
from ipywidgets import interact,interactive,fixed,widgets
from IPython.display import display,clear_output
from simplejson.decoder import JSONDecodeError
import logging
import sys
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import PatternFill
import pinnacle.enums as enums
from IPython.core.debugger import set_trace
from collections import  OrderedDict
from threading import Thread
import threading
from queue import Queue
from Classes.Comp import Comp
from HelperMethods import Methods
import os.path
import re
import numpy as np
from fuzzywuzzy import fuzz
from enum import Enum


C:\Users\Mo Ragheb\Downloads\-Python-3-Bootcamp-master\Labs\Scrapper


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#testing key
Api_key="c8b3197a8547e7235954cfb88f1fa258"

Oddsconfigfile=r"C:\Users\Mo Ragheb\Downloads\Data minings\Stats\Oddssports.xlsx"
OddsLeagues= OrderedDict()
Oddssports=OrderedDict()
Logfilelocation="C:\\Users\Mo Ragheb\\Downloads\Data minings\\Scrapper logs\\Odds"+str (time.strftime("%d-%m-%Y"+"test.txt"))
#Logfilelocation="test.txt"
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level = logging.DEBUG, format = '%(asctime)s  %(levelname)-10s %(processName)s %(funcName)s   %(message)s', filename = Logfilelocation,filemode="w" )
Eventsqueue=Queue()
Comp.buildallleagues()
DataMiningspath="C:\\Users\\Mo Ragheb\\Downloads\Data minings\\"
Trustedbookamers= OrderedDict(
{
  'ladbrokes':0,
 'betfair':1,
 'paddypower':2,
 'williamhill':3,
 'betvictor':4,
 'skybet':5,
 'mybookieag':6,
 'bovada':7,
 'bookmaker':8,
 'betonlineag':9       
}
)



In [3]:
class OddEvent:
    def __init__(self,sportid="",leagueid="",hometeam="",awayteam="",startdate=datetime.now().date(),bookmakers=[]):
        self.sportid=""
        self.leagueid=leagueid
        self.hometeam=hometeam
        self.awayteam=awayteam
        self.startdate=startdate
        self.bookmakers=[]
    def __str__(self):
       event= f"start time {str(self.startdate)} Sport {self.sportid}   League is {self.leagueid} home team is {self.hometeam} away team is {self.awayteam}"
       for bookmaker in  self.bookmakers:
            event+="\n\t"
            event+=str(bookmaker)
       return event

In [4]:
class Bookmaker:
    global bookmaers
    def __init__(self,bookmakerid="",bookmakername="",drawodds="",homemlodds="",awaymlodds="",spread="",homespread="",awayspread="",total="",totalover="",totalunder=""):
        self.bookmakerid=bookmakerid
        self.bookmakername=bookmakername
        self.homemlodds=homemlodds
        self.awaymlodds=awaymlodds
        self.drawodds=drawodds
        self.spread=spread
        self.homespread=homespread
        self.awayspread=awayspread
        self.total=total
        self.totalover=totalover
        self.totalunder=totalunder
        
    def __str__(self):
           return(f"book maker name is {self.bookmakername} home money odds are={self.homemlodds}  away moneyline odds= {self.awaymlodds} draw money line ={self.drawodds} Total is {self.total} over odds is ={self.totalover}  under odds is = {self.totalunder}  spread is ={self.spread} home spread ods is ={self.homespread}  away spread odds={self.awayspread}")

In [5]:
class BookieCoutry(Enum):
    uk="uk"
    us="us"

In [6]:
class OddsTypes(Enum):
    MoneyLine="h2h"
    Totals="totals"
    Spreads="spreads"

In [7]:
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
                return self.it.__next__()
    def __str__():
        return str(self)



In [8]:
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

In [9]:

def loop(func, n):
    """Runs the given function n times in a loop.
    """
    for i in range(n):
         func()

In [10]:
def runthreads(f, repeats=1000, nthreads=10):
    """Starts multiple threads to execute the given function multiple
    times in each thread.
    """
    # create threads
    threads = [threading.Thread(target=loop, args=(f, repeats))
               for i in range(nthreads)]

    # start threads
    for t in threads:
        t.start()

    # wait for threads to finish
    for t in threads:
        t.join()

In [11]:
def gettheremainingrequests():
        response= requests.get('https://api.the-odds-api.com/v3/sports', params={'api_key': Api_key})
        print(response.headers['x-requests-remaining'])
        return(int(response.headers['x-requests-remaining']))
    

In [12]:
gettheremainingrequests()

117


117

In [13]:
def gettheoddsAPIsports():
    OddsLeagues.clear()
    if gettheremainingrequests()>=10:
        
        sports_response = requests.get('https://api.the-odds-api.com/v3/sports', params={'api_key': Api_key})


    sports_json = json.loads(sports_response.text)

    if not sports_json['success']:
        logging.info('There was a problem with the sports request:',sports_json['msg'] )

    else:
        logging.info('Successfully got {0} sports'.format(len(sports_json['data'])))
        for sport in sports_json['data']:
           OddsLeagues[sport["key"]]=[sport["title"],sport["group"],sport["active"]]


In [14]:
def exportallsportstoCSV():
    if len(OddsLeagues.keys())==0:
        gettheoddsAPIsports()

    if os.path.exists(Oddsconfigfile):
        wb=load_workbook(filename = Oddsconfigfile)
    else:
        wb=Workbook()
    if "Odds Leagues" not in wb.get_sheet_names():
        sheet=wb.create_sheet("Odds Leagues")
    else:
        sheet=wb.active
    #sheet.title=
    sheet.cell(row=1, column=1).value ="League ID"
    sheet.cell(row=1, column=2).value ="League Name"
    sheet.cell(row=1, column=3).value ="Sport Name"
    sheet.cell(row=1, column=4).value ="active"    
    index=0
    #sheet.cell(row=1, column=3).value ="VS"
    for key, league in  OddsLeagues.items() :
           sheet.cell(row=index+2, column=1).value =key
           sheet.cell(row=index+2, column=2).value =league[0]           
           sheet.cell(row=index+2, column=3).value =league[1]
           sheet.cell(row=index+2, column=4).value =league[2] 
           index+=1
    wb.save(Oddsconfigfile)
     
        

In [15]:
def buildOddsleaguesdict():
    wb=load_workbook(filename=Oddsconfigfile)
    sheet   =wb.active
    OddsLeagues.clear()
    for row in sheet.iter_rows(row_offset=1):
            if(row[0].value!=None):  
                OddsLeagues[row[0].value]=[row[1].value,row[2].value,row[3].value]
            else:
                break
    
        

In [16]:
buildOddsleaguesdict()
OddsLeagues

OrderedDict([('americanfootball_ncaaf', ['NCAAF', 'American Football', True]),
             ('americanfootball_nfl', ['NFL', 'American Football', True]),
             ('baseball_mlb', ['MLB', 'Baseball', True]),
             ('basketball_euroleague',
              ['Basketball Euroleague', 'Basketball', True]),
             ('basketball_nba', ['NBA', 'Basketball', True]),
             ('cricket_odi', ['One Day Internationals', 'Cricket', True]),
             ('cricket_test_match', ['Test Matches', 'Cricket', True]),
             ('mma_mixed_martial_arts', ['MMA', 'Mixed Martial Arts', True]),
             ('soccer_efl_champ', ['Championship', 'Soccer - UK', True]),
             ('soccer_epl', ['EPL', 'Soccer - UK', True]),
             ('soccer_france_ligue_one',
              ['Ligue 1 - France', 'Soccer - Europe', True]),
             ('soccer_france_ligue_two',
              ['Ligue 2 - France', 'Soccer - Europe', True]),
             ('soccer_germany_bundesliga',
              ['Bu

totoalodds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': Api_key,
    'sport': "soccer_epl",
    'region': BookieCoutry.uk.value, # uk | us | au
    'mkt': "h2h" # h2h | spreads | totals
})

spreads_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': Api_key,
    'sport': "soccer_epl",
    'region': BookieCoutry.uk.value, # uk | us | au
    'mkt': OddsTypes.Spreads # h2h | spreads | totals
})

In [17]:
def getoddsforallsports():
    Eventsqueue.queue.clear()
    if len(OddsLeagues)==0:
        buildOddsleaguesdict()

   # threads = [threading.Thread(target=Fixturesgen.next(), args=(, )) for i in range(numofthreads)]
    for  key in OddsLeagues.keys():
       if key=="soccer_epl": 
           gen=getodds(key)
           try :

                runthreads(gen.__next__,len(list(gen)),4)
           except StopIteration:
                continue


In [18]:
@threadsafe_generator
def getodds(leagueid):
    
    
        
        try:
            mlodds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
            'api_key': Api_key,
            'sport': leagueid,
            'region': BookieCoutry.uk.value,
            'mkt': OddsTypes.MoneyLine.value
            })
            ml_json = json.loads(mlodds_response.text)
            if  gettheremainingrequests()>20:
                for event in ml_json["data"]:
                    tempevent=OddEvent()
                    tempevent.stardate= datetime.utcfromtimestamp (event["commence_time"]).date()
                    if tempevent.startdate< datetime.now().date() +  timedelta(days=7):
                        tempevent.leagueid= leagueid
                        tempevent.sportid=OddsLeagues[tempevent.leagueid][1]
                        tempevent.hometeam= event["teams"][0].lower()
                        tempevent.awayteam= event["teams"][1].lower() 
                        if tempevent.hometeam !=event["home_team"].lower():
                             tempevent.awayteam= event["teams"][0].lower() 
                             tempevent.hometeam= event["teams"][1].lower()    

                        for bookmaker in event["sites"]:
                            if bookmaker["site_key"] in  Trustedbookamers.keys():
                                tempbookie=Bookmaker()
                                tempbookie.bookmakerid=bookmaker["site_key"]
                                tempbookie.bookmakername=bookmaker["site_nice"]
                                if event["teams"][0].lower()==event["home_team"].lower():
                                    tempbookie.homemlodds=(bookmaker["odds"]["h2h"][0])
                                    tempbookie.awaymlodds=(bookmaker["odds"]["h2h"][1])
                                else :
                                    tempbookie.homemlodds=(bookmaker["odds"]["h2h"][1])
                                    tempbookie.awaymlodds=(bookmaker["odds"]["h2h"][0])
                                if len(bookmaker["odds"]["h2h"])==3:
                                     tempbookie.drawodds=(bookmaker["odds"]["h2h"][2])


                                tempevent.bookmakers.append(tempbookie)
                        logging.info("odds yielded for this event"+str(tempevent))
                        Eventsqueue.put(tempevent)
                        print(tempevent)
                        yield(tempevent)
                    else:
                        logging.info("event is ,ore than 7 days ahead "+str(tempevent))
            else:
              logging.CRITICAL(f"no more call remainig for this {Api_key}")
              yield None
            
        except:
                 logging.debug("no fixture found for this league "+ OddsLeagues[leagueid][0])

              

In [34]:
getoddsforallsports()

243
start time 2019-08-12 Sport Soccer - UK   League is soccer_epl home team is arsenal away team is burnley
	book maker name is Sky Bet home money odds are=1.3  away moneyline odds= 10 draw money line =5 Total is  over odds is =  under odds is =   spread is = home spread ods is =  away spread odds=
	book maker name is Bet Victor home money odds are=1.3  away moneyline odds= 12 draw money line =5.75 Total is  over odds is =  under odds is =   spread is = home spread ods is =  away spread odds=
	book maker name is Paddy Power home money odds are=1.3  away moneyline odds= 8.5 draw money line =5 Total is  over odds is =  under odds is =   spread is = home spread ods is =  away spread odds=
	book maker name is Betfair home money odds are=1.3  away moneyline odds= 13 draw money line =6 Total is  over odds is =  under odds is =   spread is = home spread ods is =  away spread odds=
	book maker name is Ladbrokes home money odds are=1.29  away moneyline odds= 9.25 draw money line =4.8 Total is 

Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-cec4be1165aa>", line 6, in loop
    func()
  File "<ipython-input-8-4d0f2bae2abd>", line 14, in __next__
    return self.it.__next__()
StopIteration
Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-cec4be1165aa>", line 6, in loop
    func()
  File "<ipython-input-8-4d0f2ba

In [21]:
def buttonclick(change):
    button.disabled=True
    t.disabled=True
    gen=getodds(leagueid=leagueidlabel.value)
    while True:
        try:

            if len(OddsLeagues) ==0:
                buildOddsleaguesdict()    
                
            print(gen.__next__())
        except StopIteration:
            break
    button.disabled=False
    t.disabled=False
def changelabelname(change):
    leagueidlabel.value=str(list(OddsLeagues.keys())[t.index])
t=widgets.Select(options=OddsLeagues.values())
t.layout.width="initial"
button=widgets.Button()
button.description="Update the odds"
button.style.button_color="skyblue"
button.on_click(buttonclick)
leagueidlabel=widgets.Label("LeagueID")
#leagueidlabel.values="gggg"
t.observe(changelabelname,names=["label"])





In [22]:
clear_output()
display(t,button,leagueidlabel)


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [19]:
def updatetheextractedodds(temp):

    while len(temp.queue)>0:
        tempevent=temp.queue.pop()
        #print(tempevent)
        #mma
        if tempevent.leagueid == "mma_mixed_martial_arts":
            updateCSVwithodds(Comp(sport="mma"),tempevent)
        else :
            updateCSVwithodds(Comp.Oddsleagues[tempevent.leagueid],tempevent)
        time.sleep(2)
        continue    
        #boxing 
      


In [20]:
def generateOddsColumsnanmes():
    for comp in Comp.Oddsleagues.values():
        addthetrustedbookiecolumns(comp)

In [21]:
def addthetrustedbookiecolumns(comp=None):

        fname=getthefilename(comp)
   # set_trace()
        if not os.path.exists(DataMiningspath+fname):
            logging.info("{0} {1} {2}fixtures not saved yet".format(comp.sport,comp.country,comp.competitionname))
            return 
        wb = load_workbook(filename = DataMiningspath+fname)
        sheet=wb.active
        sheet.column_dimensions['L'].hidden= False
        sheet.column_dimensions['M'].hidden= False
        counter=0
        if comp.sport=="mma":
            start=15
        elif comp.sport== "crckiet":
            start=19
        else:
            start =14 
        for bmaker,bmakerindex in Trustedbookamers.items() :


                    sheet.cell(row=1, column=start).value= bmaker + " Home ML"         
                    sheet.cell(row=1, column=start+1).value = bmaker + " Away ML"
                    if comp.sport=="soccer":
                      sheet.cell(row=1, column=start+2).value = bmaker + " Draw ML"
                    counter+=1 
                    start =start +2 if comp.sport!="soccer" else start +3 
        logging.info(f"{fname} openned")
        sheet=Methods.adjustwidth(sheet)
        try:  
            wb.save(DataMiningspath+fname)
           # logging.info("league {0} : {1} {2} boookie ".format(comp.sport ,str(str(counter))))
            return 
        except Exception as e:
            return(fname + "is open by other person")
        return


In [22]:
def getthefilename(comp=None):
    if comp is None : 
        logging.info("no file for this extracted odds")
        return "no file for this extracted odds"
    compname= comp.competitionname 
    if compname=="wnba" or compname!="2-bundesliga" :
        compname= Methods.replacecompetionanme(compname)   
    if comp.country=="egypt":
        fname="egyptian-"+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"
    elif comp.country=="russia":
        fname="russian-"+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"        
    elif comp.sport=="handball":
        fname="handball-"+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"
    elif comp.sport=="mma" or comp.sport=="boxing" :
        fname=  comp.sport + " " + comp.season+".xlsx"
    elif compname=="super-cup":
        fname= comp.country + " " + compname.lower().replace(" ","-").replace(":","-") + " "+comp.season+".xlsx" 
    elif compname=="premiership" and comp.sport=="rugby-union":
        fname="aviva "+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"       
    else:
        fname=compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"
    #print(fname)
    logging.info("File name is :{0}".format(fname)) 

    return fname
    

In [23]:
def updateCSVwithodds(comp=None,event=None):

    fname=getthefilename(comp)

    if not os.path.exists(DataMiningspath+fname):
        logging.info("{0} {1} {2}fixtures not saved yet".format(comp.sport,comp.country,compname))
        return comp.sport+" "+ comp.country+" "+ compname +" fixtures not saved yet"
    wb = load_workbook(filename = DataMiningspath+fname)
    sheet=wb.active
    counter=0
    ratio=2
    if comp.sport=="mma":
        start=14
    elif comp.sport== "crckiet":
        start=18
    else:
        start =13 
        if comp.sport=="soccer":
            ratio =3
    logging.info(f"{fname} openned")
    hometeam=event.hometeam.replace(" ","")
    awayteam=event.awayteam.replace(" ","")
    for index,row in enumerate(sheet.iter_rows()):

                if  index!=0 and  row[12].value!="Updated" :
                         player1text= row[1].value.lower().replace(" ","").split("(")[0].replace(".","") if "(" in row[1].value else row[1].value.lower().replace(" ","").replace(".","")
                         player2text=row[3].value.lower().replace(" ","").split("(")[0].replace(".","") if "(" in row[3].value else row[3].value.lower().replace(" ","").replace(".","")
                         
                         if fuzz.partial_ratio(player1text,hometeam)>=70 and  fuzz.partial_ratio(player2text,awayteam)>=80 :
                            for bookmaker in event.bookmakers:
                                #print(bookmaker)
                                #set_trace()
                                columnindex=start+ratio * Trustedbookamers[bookmaker.bookmakerid]
                                row[columnindex].value= bookmaker.homemlodds      
                                row[columnindex+1].value = bookmaker.awaymlodds
                                if comp.sport=="soccer":
                                    row[columnindex+2].value = bookmaker.drawodds      
                            counter+=1 
                logging.info(f"{event.leagueid} event  odds has been updated ")

                                            
    sheet=Methods.adjustwidth(sheet)
    try:  
        wb.save(DataMiningspath+fname)
        print  ("league " + comp.country +" :"+compname +" "+ str(counter)+ " matches updated")
    except Exception as e:
        return(fname + "is open by other person")

In [31]:
updatetheextractedodds()

TypeError: updatetheextractedodds() missing 1 required positional argument: 'temp'

In [35]:
tempqueue=Eventsqueue.queue.copy()
len(Eventsqueue.queue)

20

In [39]:
Comp.buildallleagues()

71

In [ ]:
while len(tempqueue)!=10:
       event= tempqueue.pop()
       if event.leagueid=="soccer_france_ligue_one":
            print(event)

In [36]:
for i in range(2):
    worker = Thread(target=updatetheextractedodds, args=(Eventsqueue,))
    worker.setDaemon(True)
    worker.start()
worker.join()
Eventsqueue.task_done()

20
19
1818

1616

1414

1212

10
9
88

66

4
3
2
1


In [25]:
generateOddsColumsnanmes()

In [24]:
Comp.Oddsleagues["soccer_spain_la_liga"].competitionname

'laliga'